In [1]:
import os
import time
import torch
from torch import nn, optim
from torch import tensor
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from skimage.filters import gabor
from skimage.feature import hog
#import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import r2_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
class FaceDataset(Dataset):
    def __init__(self, index_file):
        self.df = pd.read_csv(open(index_file))
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, i):
        Y = tensor(self.df.iloc[i][0])
        X = tensor(self.df.iloc[i][1:])
        return (X, Y)

In [17]:
train_data = FaceDataset('train.csv')
batch_size = 32
train_data_loader = DataLoader(train_data, batch_size, shuffle=True, num_workers=4)

In [18]:
class NNet(nn.Module):
    def __init__(self):
        super(NNet, self).__init__()
        self.fc1 = nn.Linear(2304, 100)
        self.fc2 = nn.Linear(100, 7)
        self.output = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = x.view(-1, 2304)
        tmp = []
        for i in range(x.shape[0]):
            image = x[i]
            image_np = image.detach().cpu().numpy().reshape(48, 48)
            gabor_real, gabor_imag = gabor(image_np, frequency=1.0)
            gabor_real = tensor(gabor_real.ravel())
            gabor_imag = tensor(gabor_imag.ravel())
            hog_ = tensor(hog(image_np))
            tmp.append(hog_)
            #tmp.append(torch.cat((image, tensor(gabor_real.ravel()), tensor(gabor_imag.ravel()))))
        f0 = torch.stack(tmp)
        f1 = F.relu(self.fc1(x))
        f2 = F.relu(self.fc2(f1))
        return self.output(f2)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#torch.cuda.set_device(0)

net = NNet()
net = net.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)
print(net)

cuda
NNet(
  (fc1): Linear(in_features=2304, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=7, bias=True)
  (output): Softmax(dim=1)
)


In [20]:
total_loss = 0
for epoch in range(10):
    for i, data in enumerate(train_data_loader, 0):
        images, classes = data
        images, classes = images.to(device).float(), classes.to(device).long()
        #images, classes = images.float(), classes.long()
        optimizer.zero_grad()
        output = net(images)
        #print(output)
        
        loss = loss_function(output, classes)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() 
        if(i % 20 == 19):
            print("After epoch {}, mini-batch {}, loss: {}".format(epoch + 1, i + 1, total_loss))
            correct = 0
            total = 0
            total_loss = 0
    total_loss = 0

After epoch 1, mini-batch 20, loss: 38.86219024658203
After epoch 1, mini-batch 40, loss: 38.618561148643494
After epoch 1, mini-batch 60, loss: 38.55501437187195
After epoch 1, mini-batch 80, loss: 38.407631635665894
After epoch 1, mini-batch 100, loss: 37.9566193819046
After epoch 1, mini-batch 120, loss: 37.63605093955994
After epoch 1, mini-batch 140, loss: 37.671406984329224
After epoch 1, mini-batch 160, loss: 37.81190752983093
After epoch 1, mini-batch 180, loss: 37.273921489715576
After epoch 1, mini-batch 200, loss: 37.80914914608002
After epoch 1, mini-batch 220, loss: 37.18520724773407
After epoch 1, mini-batch 240, loss: 37.11443817615509
After epoch 1, mini-batch 260, loss: 37.45066154003143
After epoch 1, mini-batch 280, loss: 37.052218437194824
After epoch 1, mini-batch 300, loss: 36.796424984931946
After epoch 1, mini-batch 320, loss: 36.7461941242218
After epoch 1, mini-batch 340, loss: 37.00622892379761
After epoch 1, mini-batch 360, loss: 36.88809251785278
After epoc

After epoch 5, mini-batch 600, loss: 34.587056159973145
After epoch 6, mini-batch 20, loss: 34.30345034599304
After epoch 6, mini-batch 40, loss: 34.76071286201477
After epoch 6, mini-batch 60, loss: 34.65689241886139
After epoch 6, mini-batch 80, loss: 34.061206459999084
After epoch 6, mini-batch 100, loss: 34.99410426616669
After epoch 6, mini-batch 120, loss: 34.232290863990784
After epoch 6, mini-batch 140, loss: 34.9503208398819
After epoch 6, mini-batch 160, loss: 34.98229217529297
After epoch 6, mini-batch 180, loss: 35.259790778160095
After epoch 6, mini-batch 200, loss: 34.64217269420624
After epoch 6, mini-batch 220, loss: 34.50194466114044
After epoch 6, mini-batch 240, loss: 34.60487174987793
After epoch 6, mini-batch 260, loss: 34.813679337501526
After epoch 6, mini-batch 280, loss: 34.45925176143646
After epoch 6, mini-batch 300, loss: 34.7051557302475
After epoch 6, mini-batch 320, loss: 34.64745843410492
After epoch 6, mini-batch 340, loss: 34.72675621509552
After epoch

After epoch 10, mini-batch 580, loss: 33.62198758125305
After epoch 10, mini-batch 600, loss: 34.56807458400726


In [21]:
train_data = FaceDataset('public_test.csv')
batch_size = 32
test_data_loader = DataLoader(train_data, batch_size, shuffle=False, num_workers=2)

In [22]:
actual, prediction = [], []
for i, data in enumerate(test_data_loader, 0):
    images, classes = data
    images, classes = images.to(device).float(), classes.to(device).long()
    #images, classes = images.float(), classes.long()
    actual.extend(classes.squeeze().tolist())
    
    optimizer.zero_grad()
    output = net(images)
    
    prediction.extend(torch.argmax(output, dim=1).squeeze().tolist())
    #print(output)
print(f1_score(actual, prediction, average='macro'))
print(confusion_matrix(actual, prediction))

0.31918149456256284
[[ 162    0   79  277  189   43  142]
 [  14    0   12   37    9    6   20]
 [  76    0  141  263  203   92  144]
 [  36    0   67 1153  197   50  118]
 [  90    0   83  285  368   44  227]
 [  19    0   58  143   63  347   87]
 [  60    0   63  306  171   54  460]]


In [23]:
print(np.bincount(np.array(actual)))
print(np.bincount(np.array(prediction)))

[ 892   98  919 1621 1097  717 1114]
[ 457    0  503 2464 1200  636 1198]
